In [1]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
class SmallVGGNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (height, width, depth)
		chanDim = -1

		# if we are using "channels first", update the input shape
		# and channels dimension
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
			chanDim = 1
            
		# CONV => RELU => POOL layer set
		model.add(Conv2D(32, (3, 3), padding="same",
			input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
        
		# (CONV => RELU) * 2 => POOL layer set
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(64, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
        
		# (CONV => RELU) * 3 => POOL layer set
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(Conv2D(128, (3, 3), padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))
        
		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		model.add(BatchNormalization())
		model.add(Dropout(0.5))

		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))

		# return the constructed network architecture
		return model

In [4]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [5]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("F:\\STUDY MATERIAL\\DEEP LEARNING\\keras-tutorial\\keras-tutorial\\animals")))
random.seed(42)
random.shuffle(imagePaths)

# loop over the input images
for imagePath in imagePaths:
	# load the image, resize the image to be 32x32 pixels (ignoring
	# aspect ratio), flatten the image into 32x32x3=3072 pixel image
	# into a list, and store the image in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (64, 64))
	data.append(image)

	# extract the class label from the image path and update the
	# labels list
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)
    
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

[INFO] loading images...


In [6]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.25, random_state=42)

# convert the labels from integers to vectors (for 2-class, binary
# classification you should use Keras' to_categorical function
# instead as the scikit-learn's LabelBinarizer will not return a
# vector)
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [7]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

# initialize our VGG-like Convolutional Neural Network
model = SmallVGGNet.build(width=64, height=64, depth=3,
	classes=len(lb.classes_))

In [8]:
# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.01
EPOCHS = 75
BS = 32

# initialize the model and optimizer (you'll want to use
# binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

[INFO] training network...
Epoch 1/75
70/70 [==============================] - 172s 2s/step - loss: 1.3359 - accuracy: 0.5293 - val_loss: 1.8278 - val_accuracy: 0.3147
Epoch 2/75
70/70 [==============================] - 114s 2s/step - loss: 1.0596 - accuracy: 0.5766 - val_loss: 2.1609 - val_accuracy: 0.3467
Epoch 3/75
70/70 [==============================] - 113s 2s/step - loss: 0.8987 - accuracy: 0.5924 - val_loss: 2.2223 - val_accuracy: 0.3187
Epoch 4/75
70/70 [==============================] - 113s 2s/step - loss: 0.8203 - accuracy: 0.6253 - val_loss: 1.0730 - val_accuracy: 0.4093
Epoch 5/75
70/70 [==============================] - 113s 2s/step - loss: 0.7725 - accuracy: 0.6398 - val_loss: 1.4263 - val_accuracy: 0.3693
Epoch 6/75
70/70 [==============================] - 114s 2s/step - loss: 0.7223 - accuracy: 0.6506 - val_loss: 0.8949 - val_accuracy: 0.5893
Epoch 7/75
70/70 [==============================] - 119s 2s/step - loss: 0.6932 - accuracy: 0.6614 - val_loss: 0.6959 - val_acc

70/70 [==============================] - 104s 1s/step - loss: 0.4620 - accuracy: 0.7994 - val_loss: 0.7027 - val_accuracy: 0.7307
Epoch 59/75
70/70 [==============================] - 108s 2s/step - loss: 0.4438 - accuracy: 0.8066 - val_loss: 0.5498 - val_accuracy: 0.7693
Epoch 60/75
70/70 [==============================] - 109s 2s/step - loss: 0.4527 - accuracy: 0.7976 - val_loss: 0.5197 - val_accuracy: 0.7840
Epoch 61/75
70/70 [==============================] - 122s 2s/step - loss: 0.4667 - accuracy: 0.8037 - val_loss: 0.7257 - val_accuracy: 0.7280
Epoch 62/75
70/70 [==============================] - 126s 2s/step - loss: 0.4710 - accuracy: 0.7982 - val_loss: 0.5417 - val_accuracy: 0.7653
Epoch 63/75
70/70 [==============================] - 135s 2s/step - loss: 0.4485 - accuracy: 0.8039 - val_loss: 0.6478 - val_accuracy: 0.7587
Epoch 64/75
70/70 [==============================] - 124s 2s/step - loss: 0.4518 - accuracy: 0.7962 - val_loss: 0.7512 - val_accuracy: 0.7253
Epoch 65/75
70/70 

In [10]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=lb.classes_))

# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("F:\\STUDY MATERIAL\\DEEP LEARNING\\keras-tutorial\\keras-tutorial\\My Codes\\Simple vgg\\figure.jpg")

# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save("F:\\STUDY MATERIAL\\DEEP LEARNING\\keras-tutorial\\keras-tutorial\\My Codes\\Simple vgg\\model")
f = open("F:\\STUDY MATERIAL\\DEEP LEARNING\\keras-tutorial\\keras-tutorial\\My Codes\\Simple vgg\\label", "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] evaluating network...
              precision    recall  f1-score   support

        cats       0.68      0.77      0.73       236
        dogs       0.72      0.61      0.67       236
       panda       0.90      0.92      0.91       278

    accuracy                           0.78       750
   macro avg       0.77      0.77      0.77       750
weighted avg       0.78      0.78      0.78       750

[INFO] serializing network and label binarizer...
